<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/03.%20Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-retrieval} -->

# Understanding Retrieval Question Answering

### Setup

In [2]:
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing. 

In [5]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "llmapps"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens. 

In [6]:
MODEL_NAME = "text-davinci-003"
# MODEL_NAME = "gpt-4"

Let's grab some sample data, here we will download a folder containing some markdown files from our docs.

In [7]:
!git clone https://github.com/wandb/edu.git

fatal: destination path 'edu' already exists and is not an empty directory.


In [8]:
%pip install langchain-community
%pip install "unstructured[md]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files('edu/llm-apps-course/docs_sample/')
len(documents)

11

In [10]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [11]:
# function to count the number of tokens in each document
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[430, 2560, 5125, 663, 2131, 1098, 1225, 1132, 1668, 3095, 332]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting `Markdown` without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with `Markdown`

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [12]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(103, 402)

let's look at the first section

In [13]:
Markdown(document_sections[0].page_content)

description: Explore how to use W&B Tables with this 5 minute Quickstart.                                          

Tables Quickstart                                                                                                  

The following Quickstart demonstrates how to log data tables, visualize data, and query data.                      

Select the button below to try a PyTorch Quickstart example project on MNIST data.                                 

Open In Colab                                                                                                      

 1 Log a table                                                                                                     

Follow the procedure outlined below to log a Table with W&B: 1. Initialize a W&B Run with wandb.init(). 2. Create a
wandb.Table() object instance. Pass the name of the columns in your table along with the data for the columns and  
data parameters, respectively. 3. Log the table with run.log() as a key-value pair. Provide a name for your table  
for the key, and pass the object instance of wandb.Table as the value.                                             

python run = wandb.init(project="table-test") my_table = wandb.Table(columns=["a", "b"], data=[["a1", "b1"], ["a2",
"b2"]]) run.log({"Table Name": my_table})

## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query. 

In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

/tmp/ipykernel_2249/2891843054.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [15]:
retriever = db.as_retriever(search_kwargs=dict(k=3))

In [16]:
query = "How can I share my W&B report with my team members in a public W&B project?"
docs = retriever.get_relevant_documents(query)

/tmp/ipykernel_2249/824533951.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [17]:
# Let's see the results
for doc in docs:
    print(doc.metadata["source"])

edu/llm-apps-course/docs_sample/collaborate-on-reports.md
edu/llm-apps-course/docs_sample/collaborate-on-reports.md
edu/llm-apps-course/docs_sample/teams.md


## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer. 

In [18]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [19]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict(prompt)
Markdown(response)

/tmp/ipykernel_2249/336865589.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
/tmp/ipykernel_2249/336865589.py:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


To share a report with your team members in a public W&B project, you can click the Share button on the upper right
hand corner of the report and either provide their email or copy the magic link to invite them to view the report. 
They will need to log into Weights & Biases to view the report if they are invited by email, but they will not need
to log in if they are given the magic link.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [20]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)

/tmp/ipykernel_2249/2983009556.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)


To share a W&B report with your team members in a public W&B project, you can either provide an email address or   
copy the magic link to the report. Team members who are added by email will need to log into Weights & Biases to   
view the report, while those who are given a magic link do not need to log in. You can also use the Share button on
the upper right hand corner of the report to toggle permissions between edit or view access for team members.

In [21]:
import wandb
wandb.finish()